In [9]:
import clickhouse_connect
import pandas as pd
import polars as pl
import os

In [10]:
# Set read formats to customize data output from Clickhouse
# https://clickhouse.com/docs/en/integrations/python#read-format-options-python-types
from clickhouse_connect.datatypes.format import set_read_format

# Return both IPv6 and IPv4 values as strings
set_read_format("IPv*", "string")

# Return binary as string
set_read_format("FixedString", "string")

# sets large ints to floats so that there are no large int overflow errors when converting to polars dataframe
set_read_format("Int*", "float")

In [11]:
# Create ClickHouse client
client = clickhouse_connect.get_client(
    host=os.environ.get("HOST"),
    username=os.environ.get("USERNAME"),
    password=os.environ.get("PASSWORD"),
    secure=True,
)

In [12]:
# Execute the query and return as a pandas dataframe
query = "SELECT * FROM mempool_transaction WHERE event_date_time > NOW() - INTERVAL '1 HOUR' LIMIT 1000"
results: pd.DataFrame = client.query_df(query)

In [15]:
# need to drop `meta_labels`, otherwise converting to polars will give the following error:
# ComputeError: a StructArray must contain at least one field
pl_df: pl.DataFrame = pl.from_pandas(results.drop('meta_labels', axis=1))

In [16]:
pl_df.head(5)

event_date_time,hash,from,to,nonce,gas_price,gas,gas_tip_cap,gas_fee_cap,value,type,size,call_data_size,blob_gas,blob_gas_fee_cap,blob_hashes,blob_sidecars_size,blob_sidecars_empty_size,meta_client_name,meta_client_id,meta_client_version,meta_client_implementation,meta_client_os,meta_client_ip,meta_client_geo_city,meta_client_geo_country,meta_client_geo_country_code,meta_client_geo_continent_code,meta_client_geo_longitude,meta_client_geo_latitude,meta_client_geo_autonomous_system_number,meta_client_geo_autonomous_system_organization,meta_network_id,meta_network_name,meta_execution_fork_id_hash,meta_execution_fork_id_next
datetime[ms],str,str,str,u64,i64,u64,i64,i64,f64,u8,u32,u32,u64,f64,list[str],u32,u32,str,str,str,str,str,str,str,str,str,str,f64,f64,u32,str,i32,str,str,str
2024-03-13 20:02:55.453,"""0x0e5a62cc9670…","""0xb7C435A069a6…","""0x7a4Ee6f9F0aB…",498,23609733806,162653,1500000000,23609733806,0.0,2,1467,1348,null,null,[],null,null,"""xatu-sentry-sf…","""f8440719-9e3b-…","""v0.0.154-2b9b2…","""Xatu""","""linux""","""64.23.197.136""","""San Francisco""","""United States""","""US""","""NA""",-122.4245,37.7809,14061,"""DIGITALOCEAN-A…",11155111,"""sepolia""","""0x88cf81d9""","""0"""
2024-03-13 20:02:55.764,"""0xcbb6b756c4ff…","""0x11E9CA82A3a7…","""0x2550F186feB8…",19899,1800000000000,21000,18000000000,1800000000000,0.0,3,656172,0,655360,1.8000e11,"[""0x01205e11d249ccced741227aad240fd3acba672b72af6c82ef62471d9243bd1f"", ""0x0118e2fe0a1377d29bf8942ff7acabd2a70f264e04e5a197fdb910d9b971505f"", … ""0x01e9d8f6f7c5744bbbaab0bf5e1e8760c4d3f4b17124037db25ba928e7cfbd40""]",655360,322573,"""xatu-sentry-tl…","""51c913c8-e330-…","""v0.0.154-2b9b2…","""Xatu""","""linux""","""64.176.164.164…","""""","""Israel""","""IL""","""AS""",34.7652,32.0666,20473,"""AS-CHOOPA""",17000,"""holesky""","""0x9b192ad0""","""0"""
2024-03-13 20:02:55.888,"""0xa83fe7ea16fc…","""0x38d92fA70c5a…","""0x68349A67117A…",20749,34815217116,67384,1000000000,34815217116,0.0,2,3048,2929,null,null,[],null,null,"""xatu-sentry-sf…","""f8440719-9e3b-…","""v0.0.154-2b9b2…","""Xatu""","""linux""","""64.23.197.136""","""San Francisco""","""United States""","""US""","""NA""",-122.4245,37.7809,14061,"""DIGITALOCEAN-A…",11155111,"""sepolia""","""0x88cf81d9""","""0"""
2024-03-13 20:02:56.194,"""0xa114040a6157…","""0x35d8F844BF72…","""0x4f92AD0075ca…",1,1000000000,21000,1000000000,1000000000,1.7118e14,0,107,0,null,null,[],null,null,"""xatu-sentry-sc…","""6b7dfc90-74f3-…","""v0.0.154-2b9b2…","""Xatu""","""linux""","""64.176.12.87""","""Santiago""","""Chile""","""CL""","""SA""",-70.6536,-33.4521,20473,"""AS-CHOOPA""",1,"""mainnet""","""0x9f3d2254""","""0"""
2024-03-13 20:02:56.261,"""0xf5dd8c5a00b8…","""0x57aC05B66636…","""0x6ef59533ff18…",105,34815217116,67308,1000000000,34815217116,0.0,2,3047,2931,null,null,[],null,null,"""xatu-sentry-sf…","""f8440719-9e3b-…","""v0.0.154-2b9b2…","""Xatu""","""linux""","""64.23.197.136""","""San Francisco""","""United States""","""US""","""NA""",-122.4245,37.7809,14061,"""DIGITALOCEAN-A…",11155111,"""sepolia""","""0x88cf81d9""","""0"""
